<a href="https://colab.research.google.com/github/surajsy/ColabNotebooks/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.version

'3.6.9 (default, Apr 18 2020, 01:56:04) \n[GCC 8.4.0]'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable

import scipy.io as sio
import glob
import numpy as np

In [ ]:
path = '/content/gdrive/My Drive/PRNN/MNIST/'

training_data = []
for f in glob.glob(path+'*mnist_train*',recursive=True):
  training_data.append(sio.loadmat(f))

test_data = []
for f in glob.glob(path+'*mnist_test*',recursive=True):
  test_data.append(sio.loadmat(f))

rot_train_data = []
for f in glob.glob(path+'*mnist-rot_train*',recursive=True):
  rot_train_data.append(sio.loadmat(f))

rot_test_data = []
for f in glob.glob(path+'*mnist-rot_test*',recursive=True):
  rot_test_data.append(sio.loadmat(f))

In [ ]:
train_labels = training_data[0]['training_label'].astype(np.long)
train_images = training_data[1]['training_data'].astype(np.float32)

test_labels = test_data[0]['test_label'].astype(np.long)
test_images = test_data[1]['test_data'].astype(np.float32)

rot_train_labels = rot_train_data[1]['train_label'].astype(np.long)
rot_train_images = rot_train_data[0]['train_data'].astype(np.float32)

rot_test_labels = test_data[0]['test_label'].astype(np.long)
rot_test_images = test_data[1]['test_data'].astype(np.float32)

In [ ]:
np.shape(rot_train_images)

(12000, 28, 28)

In [ ]:
input_size    = 784  
hidden_size   = 500  
num_classes   = 10   
num_epochs    = 5     
batch_size    = 100   
learning_rate = 1e-3 

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=list(zip(train_images,train_labels)),
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=list(zip(test_images,test_labels)),
                                          batch_size=batch_size,
                                          shuffle=False)

rot_train_loader = torch.utils.data.DataLoader(dataset=list(zip(rot_train_images,rot_train_labels.T)),
                                          batch_size=batch_size,
                                          shuffle=True)

rot_test_loader = torch.utils.data.DataLoader(dataset=list(zip(rot_test_images,rot_test_labels)),
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
net = Net(input_size, hidden_size, num_classes)

In [ ]:
use_cuda = True

if use_cuda and torch.cuda.is_available():
    net.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
def train_fnn(train_loader, net, num_epochs, reshape=False):
  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):
        if reshape: 
          images = Variable(images.view(-1,1,28,28))
        else:
          images = Variable(images.view(-1,784))         
        labels = Variable(labels)
          
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             
        outputs = net(images)                            
        loss = criterion(outputs, labels.squeeze())                 
        loss.backward()                                   
        optimizer.step()                                  
        
        if (i+1) % 100 == 0:                             
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                %(epoch+1, num_epochs, i+1, len(train_images)//batch_size, loss.item()))

def test_fnn(loader, net, reshape=False):              
  correct = 0
  total = 0
  for images, labels in loader:
    if reshape:
      images = Variable(images.view(-1,1,28,28))
    else:
      images = Variable(images.view(-1,784))
    
    if use_cuda and torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    
    
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    # print(predicted.size(), labels.size())                    
    correct += (predicted == labels.squeeze()).sum()     
      
  print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

In [ ]:
train_fnn(train_loader, net, num_epochs)

Epoch [1/5], Step [100/500], Loss: 0.1201
Epoch [1/5], Step [200/500], Loss: 0.0234
Epoch [1/5], Step [300/500], Loss: 0.0219
Epoch [1/5], Step [400/500], Loss: 0.0339
Epoch [1/5], Step [500/500], Loss: 0.0108
Epoch [2/5], Step [100/500], Loss: 0.0135
Epoch [2/5], Step [200/500], Loss: 0.0119
Epoch [2/5], Step [300/500], Loss: 0.0174
Epoch [2/5], Step [400/500], Loss: 0.0156
Epoch [2/5], Step [500/500], Loss: 0.0079
Epoch [3/5], Step [100/500], Loss: 0.0108
Epoch [3/5], Step [200/500], Loss: 0.0065
Epoch [3/5], Step [300/500], Loss: 0.0030
Epoch [3/5], Step [400/500], Loss: 0.0138
Epoch [3/5], Step [500/500], Loss: 0.0120
Epoch [4/5], Step [100/500], Loss: 0.0053
Epoch [4/5], Step [200/500], Loss: 0.0039
Epoch [4/5], Step [300/500], Loss: 0.0100
Epoch [4/5], Step [400/500], Loss: 0.0075
Epoch [4/5], Step [500/500], Loss: 0.0107
Epoch [5/5], Step [100/500], Loss: 0.0072
Epoch [5/5], Step [200/500], Loss: 0.0050
Epoch [5/5], Step [300/500], Loss: 0.0065
Epoch [5/5], Step [400/500], Loss:

In [ ]:
test_fnn(test_loader, net)
test_fnn(rot_test_loader, net)

Accuracy of the network on the 10K test images: 98 %
Accuracy of the network on the 10K test images: 98 %


In [ ]:
torch.save(net.state_dict(), path+'fnn_model.pkl')

In [ ]:
train_fnn(rot_train_loader, net, num_epochs)

Epoch [1/5], Step [100/500], Loss: 0.5901
Epoch [2/5], Step [100/500], Loss: 0.5078
Epoch [3/5], Step [100/500], Loss: 0.0585
Epoch [4/5], Step [100/500], Loss: 0.0133
Epoch [5/5], Step [100/500], Loss: 0.0130


In [ ]:
test_fnn(test_loader, net)
test_fnn(rot_test_loader, net)

Accuracy of the network on the 10K test images: 76 %
Accuracy of the network on the 10K test images: 76 %


In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.maxpool1 = nn.MaxPool2d(2)
        self.maxpool2 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()                    
        self.fc1 = nn.Linear(320,50)  
        self.relu = nn.ReLU()                          
        self.fc2 = nn.Linear(50, num_classes) 
    
    def forward(self, x):                              
        out = self.conv1(x)
        out = self.maxpool1(out)

        out = self.conv2(out)
        out = self.maxpool2(out)

        out = self.flatten(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
conv_net = ConvNet(num_classes)

In [ ]:
use_cuda = True

if use_cuda and torch.cuda.is_available():
    conv_net.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(conv_net.parameters(), lr=learning_rate)

In [ ]:
train_fnn(train_loader, conv_net, num_epochs, reshape=True)

Epoch [1/5], Step [100/500], Loss: 0.0205
Epoch [1/5], Step [200/500], Loss: 0.0063
Epoch [1/5], Step [300/500], Loss: 0.0211
Epoch [1/5], Step [400/500], Loss: 0.0104
Epoch [1/5], Step [500/500], Loss: 0.0028
Epoch [2/5], Step [100/500], Loss: 0.0038
Epoch [2/5], Step [200/500], Loss: 0.0068
Epoch [2/5], Step [300/500], Loss: 0.0028
Epoch [2/5], Step [400/500], Loss: 0.0010
Epoch [2/5], Step [500/500], Loss: 0.0030
Epoch [3/5], Step [100/500], Loss: 0.0077
Epoch [3/5], Step [200/500], Loss: 0.0018
Epoch [3/5], Step [300/500], Loss: 0.0012
Epoch [3/5], Step [400/500], Loss: 0.0033
Epoch [3/5], Step [500/500], Loss: 0.0008
Epoch [4/5], Step [100/500], Loss: 0.0030
Epoch [4/5], Step [200/500], Loss: 0.0027
Epoch [4/5], Step [300/500], Loss: 0.0026
Epoch [4/5], Step [400/500], Loss: 0.0001
Epoch [4/5], Step [500/500], Loss: 0.0004
Epoch [5/5], Step [100/500], Loss: 0.0046
Epoch [5/5], Step [200/500], Loss: 0.0050
Epoch [5/5], Step [300/500], Loss: 0.0016
Epoch [5/5], Step [400/500], Loss:

In [ ]:
test_fnn(test_loader, conv_net, reshape=True)
test_fnn(rot_test_loader, conv_net, reshape=True)

Accuracy of the network on the 10K test images: 98 %
Accuracy of the network on the 10K test images: 98 %


In [ ]:
train_fnn(rot_train_loader, conv_net, num_epochs, reshape=True)

Epoch [1/5], Step [100/500], Loss: 0.1132
Epoch [2/5], Step [100/500], Loss: 0.0593
Epoch [3/5], Step [100/500], Loss: 0.0088
Epoch [4/5], Step [100/500], Loss: 0.0231
Epoch [5/5], Step [100/500], Loss: 0.0035


In [ ]:
test_fnn(test_loader, conv_net, reshape=True)
test_fnn(rot_test_loader, conv_net, reshape=True)

Accuracy of the network on the 10K test images: 95 %
Accuracy of the network on the 10K test images: 95 %


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
C = [0.1, 1, 10]
k = 'rbf'

In [ ]:
rot_train = np.reshape(rot_train_images, (-1, 784))

In [ ]:
np.shape(rot_train)

(12000, 784)

In [ ]:
for c in C:
  clf = SVC(C=c, kernel=k, degree=5)
  clf.fit(train_images, train_labels)
  pred = clf.predict(test_images)
  print('non-rot: kernel:', k, '	C:', c, '	SVM Accuracy:', 100*accuracy_score(test_labels,pred))
  pred = clf.predict(rot_test_images)
  print('non-rot: kernel:', k, '	C:', c, '	SVM Accuracy:', 100*accuracy_score(rot_test_labels,pred))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
all_train = np.concatenate((train_images, rot_train), axis=0)

In [ ]:
np.shape(all_train)

(62000, 784)